In [74]:
# import libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline            

### Read data from file and create vectors
The function **get_vectors** takes a filename as input and returns a matrix X containing each row was a datapoint and vector y containing target labels.

In [75]:
def get_vectors(filename):
    try:
        f = open(filename, 'r')
    except OSError:
        print(f'{filename} could not be opened.\n')
        sys.exit()
        
    # initialize list to store feature and labels for training data
    features = []             
    labels = []
    
    with f:
        line = f.readline()
        while line != '':
            # strip newline and outer parenthesis
            line = line.strip('\n')
            line = line.strip('( )')
            
            # extrace label and append to labels list
            single_label = line.split('), ')[-1]
            labels.append(single_label)
            
            # extrace features and append to features list
            feat = line.split('), ')[0].split(', ')
            features.append(feat)
            
            # read next line
            line = f.readline()
        
        # create dataframe of features and append labels
        X = np.array(features, dtype = float, ndmin = 2)
        
        # convert labels list to array
        y = np.array(labels, dtype = str, ndmin = 2)
        
        return X, y.transpose()

### Distance Calculation

In [76]:
# This function calculates euclidean distance between training datapoints and test data point
def get_cartesian_distance(X_train, p):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    sum_of_squared_diff = np.zeros((n, 1), dtype = float)
    
    # use vectorization to get sum of squared difference
    for i in range(f_n):
        x_vector = X_train[:,i].reshape((n,1))
        sum_of_squared_diff = sum_of_squared_diff + (x_vector - p[i])**2
        
    # take sq root to get array of cartesianeuclidean distance
    euc_dist = np.sqrt(sum_of_squared_diff)
    
    return euc_dist

# This function calculates manhattan distance between training datapoints and test data point
def get_manhattan_distance(X_train, p):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    sum_of_abs_diff = np.zeros((n, 1), dtype = float)
    
    # use vectorization to get sum of squared difference
    for i in range(f_n):
        x_vector = X_train[:,i].reshape((n,1))
        sum_of_abs_diff = sum_of_abs_diff + abs(x_vector - p[i])
        
    # take sq root to get array of cartesianeuclidean distance
    man_dist = np.sqrt(sum_of_abs_diff)
    
    return man_dist


## 2 b)

### KNN Implementation 

The **predict_class_label** function takes training data(X_train, y_train), prediction datapoint(p), number of neighbors(k) to consider and distance type for similarity measurement(dist_type) as arguments and returns a prediction class based on highest posterior class probability value i.e. argmax_c P(class|< data >).

Key notes:
1. The distance, prior_class_probabilities and labels of the first 'k' neighbors are printed if verbose = 1.
2. Tie breaker: If two or more classes have the same highest probability value, class label of the closest neighbor among them
   is selected.

In [86]:

def predict_class_with_knn(X_train, y_train, p, k, dist_type, verbose = 'n'):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    if dist_type == 'cartesian':
        dist_arr = get_cartesian_distance(X_train, p)
    elif dist_type == 'manhattan':
        dist_arr = get_manhattan_distance(X_train, p)
    
    # concat with y_train labels and sort in ascending order of the distance
    dist_arr = np.concatenate((dist_arr, y_train), axis = 1)
    dist_arr = dist_arr[dist_arr[:,0].argsort()]
    
    # the first 'k' rows contain distance and labels of the k nearest neighbors
    knn = dist_arr[0:k,:]
    
    # save class labels of the k nearest neighbors as a list s.t. to count occurence
    knn_labels = list(knn[:,1])
    
    # class labels
    class_labels = list(set(knn_labels))
    
    # calculate posterior class probability
    class_probabilies = {}
    class_probabilies['Metal'] = float(knn_labels.count('Metal')/n)
    class_probabilies['Ceramic'] = float(knn_labels.count('Ceramic')/n)
    class_probabilies['Plastic'] = float(knn_labels.count('Plastic')/n)
   
    # knn_post_class_prob stores posterior probability of the 'k' nearest neighbors based on their class label
    knn_post_class_prob = []
    
    for idx, item in enumerate(knn_labels):
        knn_post_class_prob.append(class_probabilies[item]) 
    
    max_class_prob_idx = 0
    max_class_prob = float(0)
    
    # get class with highest posterior probability
    for i in range(k):

        # the class will only be considerd if its prior probability value is greater than the currrent max value
        # as we want to break any occurence of a tie based on which one is the closest neighbor
        if knn_post_class_prob[i] > max_class_prob:
            
            # value of highest posterior class probability
            max_class_prob = knn_post_class_prob[i]
            
            # index of neighbor with highest posterior class probability
            max_class_prob_idx = i
            
    # check if a max posterior probability tie had occured
    count = 0
    tie = 0
    for key, val in class_probabilies.items():
        
        # count number of max posterior probability occurences
        count = count + int(max_class_prob == val)
        
        # if there is more than one occurence of maximu posterior probability value, a tie has occured
        tie = int(count >= 2)    
            
    # print detailed results        
    if verbose == 'y':
        print(f'\nFor datapoint {p} the {k} nearest neighbor(s) are:')
        print(knn)
        print(f'The posterior class probabilities are:')
        for key, val in class_probabilies.items():
            print(f'{key}: {val:0.6f}')
        
        if tie == 0:
            print(f'Highest posterior probability is {max_class_prob:0.6f} for class {knn_labels[max_class_prob_idx]}.')
        elif tie == 1:
            print(f'Highest posterior probability is {max_class_prob:0.6f} for class {knn_labels[max_class_prob_idx]}.')
            print('Closest neighbor criterion was used to break the tie.')
        
    # return the class with maximum MLP and the probablity value
    return knn_labels[max_class_prob_idx], max_class_prob


### Leave-one-out evaluation function 

The **leave_one_out_evaluation** function takes the entire feature dataset(X) , correct data point labels(y), the number of neighbors to consider(k), verbose preference and the type of distance to use for nearest neighbor calculation(dist_type) as arguments and returns the accuracy (total correct predictions/ total datapoints). 

The datapoint ot be left out and tested is chosen according to its index (i.e. item at index 0 is left out during the first iteration and item at index n-1 is left our during the last iteration).

The **get_evaluation_results** function takes the entire feature dataset(X) , correct data point labels(y)and the type of distance to use for nearest neighbor calculation(dist_type) and verbose preference as arguments and calls **leave_one_out_evaluation** function to get accuracy values for different values of k(1, 3 and 5).

In [79]:
def leave_one_out_evaluation(X, y, k, dist_type, verbose):
    
    # get number of training items and number of features
    n, f_n = X.shape
    
    # prediction labels generated by 'predict_class_with_knn' will be stored in this list
    predictions = []
    
    for i in range(n):
        X_train = np.delete(X, i, axis = 0)
        y_train = np.delete(y, i, axis = 0)
        X_test = X[i,:]
        pred, prob = predict_class_with_knn(X_train, y_train, X_test, k, dist_type, verbose)
        predictions.append(pred)
    
    # convert prediction list to numpy array
    predictions = np.array(predictions, dtype = str, ndmin = 2)
    predictions = predictions.reshape(predictions.shape[1], 1)
    
    # return accuracy
    return (np.sum(y == predictions))/n


def get_evaluation_results(X, y, dist_type, verbose):
    # initialize distionary to store accuracy values for different 'k' values
    accuracy = {}

    # calculate accuracy for various values of 'k'
    accuracy[1] = leave_one_out_evaluation(X, y, 1, dist_type, verbose)
    accuracy[3] = leave_one_out_evaluation(X, y, 3, dist_type, verbose)
    accuracy[5] = leave_one_out_evaluation(X, y, 5, dist_type, verbose)

    print(f'\nFinal result With {dist_type} distance:')
    for key, value in accuracy.items():
        print(f'For k = {key} the accuracy is {value:0.6f}.')

### Provide training filename & prediction data point
The file must contain 1 datapoint per line in format (( height, diameter, weight, hue ), label ) which is similar to the format provided for the assignment

In [87]:
# provide training and test filename
fname_train = str(input('Enter file containing training data: '))
fname_test = str(input('Enter file containing test data: '))

# provide 'k' value
k = int(input('Enter value of k: '))

# provide verbose preference
verbose = str(input('Would you like to print detailed results?(y/n): '))
print('\n')

# get training data as vectors
X, y = get_vectors(fname_train)

# read test file to make predictions
with open(fname_test, 'r') as f:
    line = f.readline()
    i = 1
    while line != '':
        line = line.strip('\n')
        line = line.strip('( )')
        values = line.split(', ')
        p = np.array(values, dtype = float)
        predicted_class, prob_value = predict_class_with_knn(X, y, p, k, 'cartesian', verbose)
        print(f"Final Class prediction for datapoint {i} : '{predicted_class}' with a probability of {prob_value:.4f}.")
        i = i + 1
        line = f.readline()
    

Enter value of k: 3
Would you like to print detailed results?(y/n): y



For datapoint [0.12671048 0.06804045 0.20859883 3.95879103] the 3 nearest neighbor(s) are:
[['0.22991008654808265' 'Ceramic']
 ['0.40132508774163955' 'Metal']
 ['0.46414161830789835' 'Plastic']]
The posterior class probabilities are:
Metal: 0.083333
Ceramic: 0.083333
Plastic: 0.083333
Highest posterior probability is 0.083333 for class Ceramic.
Closest neighbor criterion was used to break the tie.
Final Class prediction for datapoint 1 : 'Ceramic' with a probability of 0.0833.

For datapoint [0.08007783 0.05160717 0.17838065 2.92909724] the 3 nearest neighbor(s) are:
[['0.1626000364448459' 'Ceramic']
 ['0.22205364725787488' 'Metal']
 ['0.32726564816026826' 'Ceramic']]
The posterior class probabilities are:
Metal: 0.083333
Ceramic: 0.166667
Plastic: 0.000000
Highest posterior probability is 0.166667 for class Ceramic.
Final Class prediction for datapoint 2 : 'Ceramic' with a probability of 0.1667.

For datapoint [0

## 2 c)

### K = 1, 3 and 5 using cartesian distance

### Enter filename for leave-one-out evaluation

In [88]:
fname_loo = str(input('Enter filename for leave-one-out evaluation data: '))

# provide verbose preference
verbose = str(input('Would you like to print detailed results?(y/n): '))
print('\n')

X, y = get_vectors(fname_loo)
get_evaluation_results(X, y, 'cartesian', verbose)

Enter filename for leave-one-out evaluation data: Data\\2_c_d_e.txt
Would you like to print detailed results?(y/n): y



For datapoint [0.10333502 0.07922548 0.18783852 2.78657642] the 1 nearest neighbor(s) are:
[['0.08576818981901319' 'Metal']]
The posterior class probabilities are:
Metal: 0.008403
Ceramic: 0.000000
Plastic: 0.000000
Highest posterior probability is 0.008403 for class Metal.

For datapoint [0.1859912  0.14277373 0.60851886 5.64781922] the 1 nearest neighbor(s) are:
[['0.1537564340952132' 'Ceramic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.008403
Plastic: 0.000000
Highest posterior probability is 0.008403 for class Ceramic.

For datapoint [0.06690281 0.04501028 0.11844696 2.78001842] the 1 nearest neighbor(s) are:
[['0.08576818981901319' 'Plastic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.000000
Plastic: 0.008403
Highest posterior probability is 0.008403 for class Plastic.

For datapoint [0.12899039 0.15       0.43308


For datapoint [0.17070678 0.14826808 0.48568546 4.24761684] the 1 nearest neighbor(s) are:
[['0.05524288246583025' 'Plastic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.000000
Plastic: 0.008403
Highest posterior probability is 0.008403 for class Plastic.

For datapoint [0.14295041 0.12722081 0.34624317 4.21255473] the 1 nearest neighbor(s) are:
[['0.09384252283517042' 'Ceramic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.008403
Plastic: 0.000000
Highest posterior probability is 0.008403 for class Ceramic.

For datapoint [0.11116157 0.07605591 0.26277683 2.89843362] the 1 nearest neighbor(s) are:
[['0.060200644792696455' 'Ceramic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.008403
Plastic: 0.000000
Highest posterior probability is 0.008403 for class Ceramic.

For datapoint [0.11618069 0.08174889 0.28402569 1.64666434] the 1 nearest neighbor(s) are:
[['0.15550300331607605' 'Plastic']]
The posterior class probabiliti

Highest posterior probability is 0.016807 for class Ceramic.

For datapoint [0.05       0.06175543 0.1699843  2.8917648 ] the 3 nearest neighbor(s) are:
[['0.04801439965945433' 'Ceramic']
 ['0.10632874409166383' 'Metal']
 ['0.11225046935979778' 'Plastic']]
The posterior class probabilities are:
Metal: 0.008403
Ceramic: 0.008403
Plastic: 0.008403
Highest posterior probability is 0.008403 for class Ceramic.
Closest neighbor criterion was used to break the tie.

For datapoint [0.16430498 0.15       0.6332237  3.40106854] the 3 nearest neighbor(s) are:
[['0.059332905787719055' 'Plastic']
 ['0.10630936763571529' 'Plastic']
 ['0.11635696368824747' 'Plastic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.000000
Plastic: 0.025210
Highest posterior probability is 0.025210 for class Plastic.

For datapoint [0.07341828 0.03       0.10507327 3.14439812] the 3 nearest neighbor(s) are:
[['0.14773259564106694' 'Ceramic']
 ['0.1571869876463072' 'Plastic']
 ['0.16378998932811925' '

For datapoint [0.17250305 0.15       0.60605286 4.43825807] the 5 nearest neighbor(s) are:
[['0.12459282863575002' 'Metal']
 ['0.20553566106460427' 'Metal']
 ['0.2172860849784545' 'Ceramic']
 ['0.22547419951561587' 'Plastic']
 ['0.23056125060656743' 'Ceramic']]
The posterior class probabilities are:
Metal: 0.016807
Ceramic: 0.016807
Plastic: 0.008403
Highest posterior probability is 0.016807 for class Metal.
Closest neighbor criterion was used to break the tie.

For datapoint [0.06898283 0.080258   0.3400786  2.80103453] the 5 nearest neighbor(s) are:
[['0.08004365695207447' 'Plastic']
 ['0.12104950732124137' 'Plastic']
 ['0.12233288022592617' 'Ceramic']
 ['0.1313729381542782' 'Plastic']
 ['0.1567393273416478' 'Plastic']]
The posterior class probabilities are:
Metal: 0.000000
Ceramic: 0.008403
Plastic: 0.033613
Highest posterior probability is 0.033613 for class Plastic.

For datapoint [0.16369468 0.10495649 0.47624154 3.28185761] the 5 nearest neighbor(s) are:
[['0.09985252184559035' 

### Conclusion: k = 1 was found to give the best performance in terms of accuracy.

## 2 d) 

### K = 1, 3 and 5 using manhattan distance

In [16]:
get_evaluation_results(X, y, 'manhattan', verbose)

With manhattan distance:
For k = 1 the accuracy is 0.500000.
For k = 3 the accuracy is 0.458333.
For k = 5 the accuracy is 0.516667.


### Conclusion: Manhattan distance was found to perform better than cartesian distance. Accuracy is highest for a value of k = 5.

## 2 e) 

### Prediction after removing 4th attribute (hue) 

In [10]:
# remove 4th attribute from X
X_3 = np.delete(X, 3, axis = 1)

# check shape of x
X_3.shape

get_evaluation_results(X_3, y, 'cartesian', verbose)

### Conclusion: Removing the 4th attribute was found to significantly improve accuracy with highest accuracy value for k = 3